# Embeddings(Text, Audio)


In [1]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

print(torch.__version__)

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [2]:
import pickle
# Kemdy19 wav, txt, eda, temp 데이터를 정리한 것
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl', 'rb') as f:
    annotation_19_nonmissing = pickle.load(f)

In [3]:
annotation_19_nonmissing

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 9, 0, 0, 0, 1]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]","[4.408951, 4.403825, 4.410233, 4.421767, 4.429...","[34.66, 34.66, 34.66, 34.66, 34.66, 34.68, 34....",33,33,"[-0.926455937246618, -0.9285481420682529, -0.9...","[1.6900874192514124, 1.6900874192514124, 1.690..."
1,Sess01_script01_F001,fear,1.5,3.6,"[1, 0, 2, 0, 0, 0, 7]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]","[0.145914, 0.140794, 0.144634, 0.145914, 0.140...","[30.69, 30.69, 30.69, 30.69, 30.71, 30.71, 30....",17,17,"[-1.407155706494133, -1.4312942323175069, -1.4...","[-1.1135528407658513, -1.1135528407658513, -1...."
2,Sess01_script01_M002,angry,1.3,4.3,"[0, 0, 0, 0, 10, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]","[4.478828, 4.396809, 4.334012, 4.322478, 4.346...","[34.61, 34.61, 34.61, 34.61, 34.61, 34.61, 34....",27,27,"[-0.8979352590723317, -0.9314117606848334, -0....","[1.6164861115016742, 1.6164861115016742, 1.616..."
3,Sess01_script01_M003,angry,1.5,3.8,"[0, 0, 1, 0, 9, 0, 0]","[6, 3, 1, 0, 0]","[1, 0, 0, 8, 1]","[4.746299, 4.733483, 4.716823, 4.702725, 4.692...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....",10,10,"[-0.7887655134914608, -0.7939964337009954, -0....","[1.5281645422019798, 1.5281645422019798, 1.528..."
4,Sess01_script01_F002,fear,1.6,3.7,"[0, 0, 2, 0, 0, 0, 8]","[5, 4, 1, 0, 0]","[1, 0, 1, 7, 1]","[0.134394, 0.135674, 0.134394, 0.134394, 0.133...","[30.69, 30.69, 30.71, 30.71, 30.71, 30.71, 30....",13,13,"[-1.461467389596724, -1.4554327581408808, -1.4...","[-1.1135528407658513, -1.1135528407658513, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,Sess20_impro04_M018,sad,2.3,2.1,"[3, 0, 0, 0, 0, 7, 0]","[2, 4, 3, 1, 0]","[0, 9, 1, 0, 0]","[0.550024, 0.547461, 0.54618, 0.543617, 0.5461...","[31.27, 31.27, 31.27, 31.27, 31.27, 31.27, 31....",31,31,"[-1.0596542781342695, -1.091127738705599, -1.1...","[-0.12418082175855716, -0.12418082175855716, -..."
9004,Sess20_impro04_F023,sad,2.5,2.3,"[2, 0, 0, 0, 0, 8, 0]","[0, 6, 3, 1, 0]","[1, 5, 4, 0, 0]","[0.216838, 0.220682, 0.221964, 0.223245, 0.221...","[29.77, 29.77, 29.77, 29.77, 29.77, 29.77, 29....",46,46,"[-0.8593367498709915, -0.7994211382956963, -0....","[-1.3249127202479716, -1.3249127202479716, -1...."
9005,Sess20_impro04_M019,sad,2.1,1.8,"[1, 0, 0, 0, 0, 9, 0]","[4, 2, 3, 1, 0]","[3, 6, 1, 0, 0]","[0.519269, 0.517987, 0.519269, 0.517987, 0.516...","[31.31, 31.31, 31.31, 31.31, 31.29, 31.29, 31....",56,56,"[-1.437323525060235, -1.4530663953108964, -1.4...","[0.697453930425023, 0.697453930425023, 0.69745..."
9006,Sess20_impro04_F024,happy,3.9,3.2,"[3, 7, 0, 0, 0, 0, 0]","[0, 0, 1, 9, 0]","[0, 1, 6, 3, 0]","[0.207868, 0.207868, 0.210431, 0.211712, 0.212...","[29.73, 29.73, 29.73, 29.73, 29.73, 29.73, 29....",52,52,"[-0.9991502347384216, -0.9991502347384216, -0....","[-1.6382728540476073, -1.6382728540476073, -1...."


In [4]:
file_lst = annotation_19_nonmissing['Segment ID']

In [5]:
file_lst

0       Sess01_script01_M001
1       Sess01_script01_F001
2       Sess01_script01_M002
3       Sess01_script01_M003
4       Sess01_script01_F002
                ...         
9003     Sess20_impro04_M018
9004     Sess20_impro04_F023
9005     Sess20_impro04_M019
9006     Sess20_impro04_F024
9007     Sess20_impro04_M020
Name: Segment ID, Length: 9008, dtype: object

## Audio Embedding

In [6]:
wav_file_lst = []
for segment in file_lst:
    session, _, _ = segment.split('_')
    session = session[-2:]

    wav_file_lst.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy19/Session{session}/' + segment + '.wav')
wav_file_lst = pd.Series(wav_file_lst)
sampling_rate = 16000


In [7]:
torch.cuda.is_available()

True

In [8]:
check_point_wav_model = 'facebook/data2vec-audio-large-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)
model_wav_cuda = Data2VecAudioModel.from_pretrained(check_point_wav_model).to('cuda')

Some weights of the model checkpoint at facebook/data2vec-audio-large-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/data2vec-audio-large-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

### audio embedding method

In [28]:
def session_check(segment_id, sess):
    if sess in segment_id:
        return True
    
    else:
        return False

# padding: https://huggingface.co/docs/transformers/pad_truncation
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_array = [dataset[i]['wav']['array'] for i in range(len(wav_file_lst))]
    inputs = processor(wav_array, sampling_rate = sampling_rate, return_tensors = 'pt', padding = 'max_length', max_length= 48000, truncation = True)
    with torch.no_grad():
        output = model_wav(**inputs)
    return output

def make_wav_embedding_pickle(file_lst, processor, model, sess_lst):
    
    for session in tqdm(sess_lst):
        wav_embeddings = wav_embeddings_func(file_lst[file_lst.apply(session_check, sess=session)], processor, model)
        wav_embeddings = wav_embeddings['last_hidden_state']
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19/paradeigma_KEMDY19_wav_embedding_{session}.pkl', 'wb') as f:
            pickle.dump(wav_embeddings, f, pickle.HIGHEST_PROTOCOL)

        del wav_embeddings
        torch.cuda.empty_cache()
        
def load_wav_embedding_pickle(path_list):
    result = {}
    for path in path_list:
        session = path[-10:-4]

        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19/{path}', 'rb') as f:
            wav_embedding = pickle.load(f)
            
        result[session] = wav_embedding
        
    return result

In [36]:
sess_lst = annotation_19_nonmissing['Segment ID'].apply(lambda x: x[:6]).unique()

make_wav_embedding_pickle(wav_file_lst, processor, model_wav, sess_lst)

100%|██████████| 20/20 [1:51:14<00:00, 333.71s/it]


In [29]:
import os

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19'
file_list = os.listdir(dir_path)
wav_path_list = sorted([file for file in file_list if 'KEMDY19_wav' in file ])

embeddings_wav_dict = load_wav_embedding_pickle(wav_path_list)

## Text Embedding
- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [13]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import sentencepiece as spm
import pandas as pd
import re


In [11]:
# text data loading
txt_file_lst = []
for segment in file_lst:
    session = segment[4:6]
    txt_file_lst.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy19/Session{session}/' + segment + '.txt')
                        
txt_file_lst = pd.Series(txt_file_lst)

In [16]:
checkpoint = 'Junmai/KR-Data2VecText-v1'
model_txt = AutoModel.from_pretrained(checkpoint) 
model_txt_cuda = AutoModel.from_pretrained(checkpoint).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model_txt.resize_token_embeddings(len(tokenizer))

Embedding(35002, 768)

### text embedding method

In [20]:
def session_check(segment_id, sess):
    if sess in segment_id:
        return True
    
    else:
        return False

def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    
    inputs = tokenizer(sentences, padding='max_length', max_length = 80, truncation=True, return_tensors="pt")
    with torch.no_grad():
        txt_embeddings= model_txt(**inputs)
    return txt_embeddings['last_hidden_state']

def make_txt_embedding_pickle(file_lst, tokenizer, model, sess_lst):
    
    for session in tqdm(sess_lst):
        txt_embeddings = text_embeddings_func(file_lst[file_lst.apply(session_check, sess=session)], tokenizer, model)
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19/paradeigma_KEMDY19_txt_embedding_{session}.pkl', 'wb') as f:
            pickle.dump(txt_embeddings, f, pickle.HIGHEST_PROTOCOL)
            
        del txt_embeddings
        torch.cuda.empty_cache()
    
def load_txt_embedding_pickle(path_list):
    result = {}
    for path in path_list:
        session = path[-10:-4]
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19/{path}', 'rb') as f:
            txt_embedding = pickle.load(f)
        result[session] = txt_embedding
        
    return result

In [21]:
sess_lst = annotation_19_nonmissing['Segment ID'].apply(lambda x: x[:6]).unique()

make_txt_embedding_pickle(txt_file_lst, tokenizer, model_txt_cuda, sess_lst)

100%|██████████| 20/20 [14:55<00:00, 44.77s/it]


In [24]:
import os

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY19'
file_list = os.listdir(dir_path)
txt_path_list = sorted([file for file in file_list if 'KEMDY19_txt' in file ])

embeddings_txt_dict = load_txt_embedding_pickle(txt_path_list)


In [30]:
embedding_for_dataset = [embeddings_wav_dict, embeddings_txt_dict]

dict_keys(['Sess01', 'Sess02', 'Sess03', 'Sess04', 'Sess05', 'Sess06', 'Sess07', 'Sess08', 'Sess09', 'Sess10', 'Sess11', 'Sess12', 'Sess13', 'Sess14', 'Sess15', 'Sess16', 'Sess17', 'Sess18', 'Sess19', 'Sess20'])

In [65]:
import pickle
# Kemdy 19 임베딩 한 것
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl', 'wb') as f:
    pickle.dump(embedding_for_dataset, f, pickle.HIGHEST_PROTOCOL)


In [66]:
import pickle
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl', 'rb') as f:
    embedding_for_dataset = pickle.load(f)
